In [1]:
import coolingFunction

In [2]:
import copy
import numpy as np
import scipy
import sys
# import py2tex.py2tex as py2tex
import verdict
import os
import unyt

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patheffects as path_effects
import matplotlib.cm as cm
import matplotlib.colors as plt_colors
import matplotlib.gridspec as gridspec
import matplotlib.transforms
import palettable

In [4]:
import linefinder.analyze_data.worldlines as a_worldlines
import linefinder.analyze_data.worldline_set as worldline_set
import linefinder.analyze_data.plot_worldlines as p_worldlines
import linefinder.utils.presentation_constants as p_constants

In [5]:
import galaxy_dive.analyze_data.ahf as analyze_ahf
import galaxy_dive.plot_data.ahf as plot_ahf
import galaxy_dive.analyze_data.particle_data as particle_data
import galaxy_dive.plot_data.generic_plotter as generic_plotter
import galaxy_dive.plot_data.plotting as plotting
import galaxy_dive.utils.data_operations as data_operations
import galaxy_dive.utils.executable_helpers as exec_helpers

In [6]:
import linefinder.utils.file_management as file_management
import linefinder.config as config

# Load Data

In [7]:
sim_name = 'm12i_md'
sim_names = [
    'm11d_md',
    'm12i_md',
    'm12b_md',
#     'm12m',
#     'm12f',
]
snum = 600
tables_dir = '/work/03057/zhafen/CoolingTables/'

In [8]:
galdef = ''

In [9]:
file_manager = file_management.FileManager( 'hot_halo_physics' )

In [10]:
defaults, variations = file_manager.get_linefinder_analysis_defaults_and_variations(
    '_hothaloacc',
    sim_names = sim_names,
    galdef = galdef,
)
defaults, variations

({'data_dir': '/scratch/03057/zhafen/linefinder_data/core/m12i_res7100/data',
  'tag': 'm12i_hothaloacc',
  'halo_data_dir': '/scratch/03057/zhafen/core/m12i_res7100/halo',
  'ahf_index': 600,
  'main_halo_id': 0},
 {'m11d_md': {'data_dir': '/scratch/03057/zhafen/linefinder_data/metal_diffusion/m11d_res7100/data',
   'tag': 'm11d_md_hothaloacc',
   'halo_data_dir': '/scratch/03057/zhafen/metal_diffusion/m11d_res7100/halo',
   'ahf_index': 600,
   'main_halo_id': 0},
  'm12i_md': {'data_dir': '/scratch/03057/zhafen/linefinder_data/metal_diffusion/m12i_res7100/data',
   'tag': 'm12i_md_hothaloacc',
   'halo_data_dir': '/scratch/03057/zhafen/metal_diffusion/m12i_res7100/halo',
   'ahf_index': 600,
   'main_halo_id': 0},
  'm12b_md': {'data_dir': '/scratch/03057/zhafen/linefinder_data/metal_diffusion/m12b_res7100/data',
   'tag': 'm12b_md_hothaloacc',
   'halo_data_dir': '/scratch/03057/zhafen/metal_diffusion/m12b_res7100/halo',
   'ahf_index': 600,
   'main_halo_id': 0}})

In [11]:
w_set = worldline_set.WorldlineSet( defaults, variations )

In [12]:
ind = defaults['ahf_index'] - snum

In [13]:
w = w_set[sim_name].data_object

In [14]:
w.retrieve_halo_data()

Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [15]:
print( w.tag )

m12i_md_hothaloacc


In [16]:
m_plot_label  = r'$M_{\rm h} = 10^{' + '{:.02g}'.format( np.log10( w.m_vir[snum] ) )
m_plot_label += '} M_\odot$'
plot_label = m_plot_label + ', z={:.02}'.format( w.redshift[snum] )
print( plot_label )

$M_{\rm h} = 10^{12} M_\odot$, z=0.0


In [17]:
classification_list = copy.copy( p_constants.CLASSIFICATIONS_CGM_FATE )

In [18]:
w_plotter = p_worldlines.WorldlinesPlotter( w, label=plot_label )

## Calculate $\theta$
Also called $\phi$...

In [19]:
for sim_name_j, w_j in w_set.items():
    
    print( 'Calculating angular momentum for {}'.format( sim_name_j ) )
    
    s_data = particle_data.ParticleData(
        sdir = file_manager.get_sim_dir( sim_name_j ),
        snum = snum,
        ptype = config.PTYPE_STAR,
        halo_data_dir = file_manager.get_halo_dir( sim_name_j ),
        main_halo_id = config.MAIN_MT_HALO_ID[sim_name_j],    
    )
    
    w_j.data_object.calc_abs_phi( normal_vector=s_data.total_ang_momentum )


Calculating angular momentum for m11d_md


NameError: No files (set()) to load in /scratch/03057/zhafen/metal_diffusion/m11d_res7100/halo

## Calculate overall gas angular momentum

In [ ]:
g_data = particle_data.ParticleData(
    sdir = file_manager.get_sim_dir( sim_name ),
    snum = snum,
    ptype = config.PTYPE_GAS,
    halo_data_dir = file_manager.get_halo_dir( sim_name ),
    main_halo_id = config.MAIN_MT_HALO_ID[sim_name],    
)

g_data.calc_ang_momentum( specific_ang_momentum=True )

In [ ]:
g_plotter = generic_plotter.GenericPlotter( g_data )

## Calculate Cooling Luminosity

for sim_name_j, w_j in w_set.items():

    w_j_d = w_j.data_object
    n_H = w_j_d.get_data( 'HDen' ) * unyt.cm**-3
    rho = w_j_d.get_data( 'Den' ) * 1e10 * unyt.msun / unyt.kpc**3.
    Lcool = []
    for k, z in enumerate( w_j_d.redshift ):

        # Out of table bounds
        if z > 8.9:
            Lcool_z = np.full( n_H.shape[0], np.nan )
        else:
            # Edge case for interpolation
            if np.isclose( z, 0.0 ):
                z = 0.0000000001

            Lcool_z = (
                n_H[:,k]**2.
                * coolingFunction.LambdaFunc( z, tables_dir, 0.2485)(
                    w_j_d.get_data( 'T' )[:,k],
                    n_H[:,k],
                    w_j_d.get_data( 'Z' )[:,k],
                ) * unyt.erg / unyt.s * unyt.cm**3.
                / rho[:,k]
            ).to( 'erg/g/s').value

        Lcool.append( Lcool_z )

    w_j_d.data['lcool'] = np.array(Lcool).transpose()

## Calculate Compressional Heating

%matplotlib inline


# This depends on the time derivative of density, so we're going to plot that first to make sure it's okay...
fig = plt.figure( figsize=(11,8), facecolor='w' )
ax = plt.gca()

gs = gridspec.GridSpec( 2, 1 )

# Density
ax1 = plt.subplot( gs[0,0] )

ax1.plot(
    w.get_data( 'time' ),
    w.get_data( 'Den' )[k],
    color = 'k',
    linewidth = 3,
)

ax1.set_yscale( 'log' )

# Gradient of Density
ax2 = plt.subplot( gs[1,0] )

ax2.plot(
    w.get_data( 'time' ),
    np.gradient( w.get_data( 'Den' )[k], w.get_data( 'time' ), ),
    color = 'k',
    linewidth = 3,
)

ax2.set_yscale( 'log' )

Looks like the time derivative will be pretty irregular, as expected...

for sim_name_j, w_j in w_set.items():

    w_j = w_j.data_object

    # Calculate compressional heating
    t_second = ( w_j.get_data( 'time' ) * unyt.Gyr ).to( 'Gyr' ).value
    drhodt = np.array(
        [ np.gradient( w_j.get_data( 'HDen' )[k], t_second, ) for k in range( w_j.n_particles ) ]
    ) * unyt.cm**-3. / unyt.Gyr
    pressure = (
        2.3 *
        unyt.kb *
        w_j.get_data( 'HDen' ) * unyt.cm**-3 *
        w_j.get_data( 'T' ) * unyt.K
    )
    den = ( w_j.get_data( 'HDen' ) / unyt.cm**3 )
    w_j.data['lcompheat'] = ( np.abs( pressure * drhodt / den**2. ) / (0.7**-1*unyt.mp) ).to( 'erg/g/s' )

# Hot Mode Accretion Investigation

In [ ]:
# Setup axes
t_window = 1.
t = w.get_data( 'time' )
x_range = [ t[ind] - t_window, t[ind] ]

In [ ]:
t_snaps = t[( t > x_range[0] ) & ( t < x_range[1] )][::-1]

In [ ]:
n_snaps = t_snaps.size

In [ ]:
dt = t_snaps[1:] - t_snaps[:-1]

In [ ]:
t_bins = np.zeros( ( t_snaps.size + 1, ) )
t_bins[1:-1] = t_snaps[:-1] + dt / 2.
t_bins[0] = t_snaps[0] - dt[0] / 2.
t_bins[-1] = t_snaps[-1] + dt[-1] / 2.

In [ ]:
def get_RMLt_at_Tcool( w ):
    
    w.data_masker.clear_masks()
    w.data_masker.mask_data( 'PType', data_value=0 )

    # Median and interval stats
    logT = np.log10( w.get_selected_data( 'T', compress=False ) )#[:,ind:ind+n_snaps+1]
    R = w.get_selected_data( 'R', compress=False )#[:,ind:ind+n_snaps+1]
    L = w.get_selected_data( 'Lmag', compress=False )#[:,ind:ind+n_snaps+1]
    M = w.get_selected_data( 'M', compress=False )#[:,ind:ind+n_snaps+1]

    logT_med = np.nanmedian( logT, axis=0 )
    R_med = np.nanmedian( R, axis=0 )

    logT_low = np.nanpercentile( logT, 16, axis=0 )
    logT_high = np.nanpercentile( logT, 84, axis=0 )

    R_low = np.nanpercentile( R, 16, axis=0 )
    R_high = np.nanpercentile( R, 84, axis=0 )

    inds = []
    for logT_arr in logT:

        ind_ = -1
        for i in range( logT_arr.size ):
            if logT_arr[i] > 5.:
                ind_ = i
                break

        inds.append( ind_ )

    inds = np.array( inds )

    R_at_Tcool = np.array( [ R[i, ind] for i, ind in enumerate( inds ) ] )
    M_at_Tcool = np.array( [ M[i, ind] for i, ind in enumerate( inds ) ] )
    L_at_Tcool = np.array( [ L[i, ind] for i, ind in enumerate( inds ) ] )

    t_at_Tcool = np.array( [ t[ind] for ind in inds ] )
    
    return R_at_Tcool, M_at_Tcool, L_at_Tcool, t_at_Tcool, inds

In [ ]:
RMLt_at_Tcool = {}
for sim_name_j, w_j in w_set.items():
    
    print( 'Calculating values at T=1e5K for {}'.format( sim_name_j ) )
    
    RMLt_at_Tcool[sim_name_j] = get_RMLt_at_Tcool( w_j.data_object )

In [ ]:
R_at_Tcool, M_at_Tcool, L_at_Tcool, t_at_Tcool, inds = RMLt_at_Tcool[sim_name]

## Distribution Plots

In [ ]:
j_data = np.load( '../data/for_Zach.npz' )

In [ ]:
def find_q_level( counts, q, start=1 ):
    
    tot = counts.sum()
    def fn_to_minimize( level ):
        contained = counts[counts>level].sum()
        return ( contained/tot - q/100. )**2.
    result = scipy.optimize.minimize( fn_to_minimize, start, method='Powell' )
        
    print( 'Found contour {} to within {}'.format( q, result.fun ) )
    assert result.fun < 0.01
    
    return result.x

In [ ]:
colormap = palettable.cartocolors.qualitative.Vivid_4.mpl_colors
colors = {}
for j, sim_name_j in enumerate( sim_names ):
    colors[sim_name_j] = colormap[j]

In [ ]:
%matplotlib inline

fig = plt.figure( figsize=(10, 6 ), facecolor='w' )
ax = plt.gca()

color_objects = []
labels = []
for j, (sim_name_j, w_j) in enumerate( w_set.items() ):
    
    w_j = w_j.data_object
    
    print( 'Plotting contour for {}'.format( sim_name_j ) )
    
    R_at_Tcool, M_at_Tcool, L_at_Tcool, t_at_Tcool, inds = RMLt_at_Tcool[sim_name_j]
        
    # Make the histogram
    counts, xbins, ybins, image = ax.hist2d(
        R_at_Tcool,
        L_at_Tcool / M_at_Tcool,
        bins = [ np.logspace( 0., np.log10( w_j.r_vir[snum] ), 32 ), np.logspace( 2., 4.5, 32 ) ],
        cmap = cm.cubehelix_r,
    #     norm = plt_colors.LogNorm(),
    )

    # Contour plot grid
    logxbins = np.log10(xbins)
    xbin_centers = 10.**( logxbins[:-1] + 0.5 * ( logxbins[1] - logxbins[0] ) ) 
    logybins = np.log10(ybins)
    ybin_centers = 10.**( logybins[:-1] + 0.5 * ( logybins[1] - logybins[0] ) ) 
    meshed_xs, meshed_ys = np.meshgrid( xbin_centers, ybin_centers )

    # Contour plot
    percentile_levels = [ 90, 50, ]
    color = colors[sim_name_j]
    levels = [ find_q_level( counts, q, start=1, ) for q in percentile_levels ]
    CS = ax.contour(
        meshed_xs, meshed_ys,
        counts.transpose(),
        extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],
        linewidths = [ 2.5, 5, ],
        levels = levels,
        colors = [ color, ]*len( levels ),
        alpha = 0.8,
    )
    
    # For legend
    color_object = matplotlib.patches.Rectangle(                         
        (0, 0),                                                          
        1,                                                               
        1,                                                               
        fc = color,                                 
        ec = color,                                 
    )
    color_objects.append( color_object )                                 
    labels.append( sim_name_j )    

print( 'Plotting unselected angular momentum' )
out = g_plotter.statistic_and_interval(
    'R',
    'Lmag',
    ax = ax,
    lower_percentile = 5,
    upper_percentile = 95,
    bins = xbins,
    return_values = True,
    fix_invalid = True,
    alpha = 0.2,
    linestyle = '--',
    linewidth = 2,
)


#     ax.clabel( CS, levels )

# Don't actually plot the histograms
for child in ax.get_children():
    if isinstance( child, matplotlib.collections.QuadMesh ):
        child.remove()

ax.set_xscale( 'log' )
ax.set_yscale( 'log' )
# ax.set_xlim( 0, w.r_vir[snum] )
# ax.set_ylim( 1e2, 3e4 )

# ax.scatter(
#     R_at_Tcool,
#     L_at_Tcool / M_at_Tcool, 
#     s = 1,
#     alpha = 0.5,
# )

ax.plot(
    j_data['r'],
    j_data['r'] * j_data['vc'],
    color = 'k',
)

# j_data_ind = 245
j_data_ind = 65
ax.annotate(
    s=r'$v_{\rm c}r$',
    xy=( j_data['r'][j_data_ind], ( j_data['r'] * j_data['vc'] )[j_data_ind],),
    xytext=(10,-10),
    xycoords='data',
    textcoords='offset points',
    fontsize=28,
)

ax.legend(
    color_objects,
    labels,
    prop={'size': 18},
    loc = 'lower right',
)

ax.set_xlabel( r'$R_{T=10^5 {\rm K}}$ (kpc)', fontsize=22 )
ax.set_ylabel( r'$j(R_{T=10^5 {\rm K}})$ (kpc km/s)', fontsize=22 )

plotting.save_fig(
    out_dir = '../figures',
    save_file = 'j_vs_rcondense.png',
    fig = fig,
)

In [ ]:
dt_before = -2.
dt_after = 0.5
color_dt = 0.2
n_particles = 10
x_lim = np.array( [ 0, 105 ] )
y_lim = np.array( [ 5e3, 3e6 ] )
y2_lim = np.array( [ 1, 1e2 ] )

In [ ]:
for j, (sim_name_j, w_j) in enumerate( w_set.items() ):
    
    print( 'Plotting streamlines for {}'.format( sim_name_j ) )
    
    # Load sim data
    R_at_Tcool, M_at_Tcool, L_at_Tcool, t_at_Tcool, inds = RMLt_at_Tcool[sim_name_j]
    r_vir = w_j.data_object.r_vir[snum]
    
    w_j.data_object.data_masker.clear_masks()
    
    # Only include particles that have never left the main galaxy
    w_j.data_object.data_masker.mask_data( 'n_out', -1, 1 )

    np.random.seed( 2 )

    fig = plt.figure( figsize=(12, 11), facecolor='w' )
    ax = plt.gca()

    gs = gridspec.GridSpec(7, 1)
    gs.update( hspace=0.001 )

    ax1 = plt.subplot(gs[:2,0])
    
    r_for_hist = copy.copy( R_at_Tcool )
    r_for_hist[r_for_hist>x_lim[1]] = x_lim[1]*.99

    n, bins, patches = ax1.hist(
        R_at_Tcool,
        bins = np.linspace( 0., 2.*r_vir, 256 ),
        color = colors[sim_name_j],
        density = True,
    )
    
    print( 'Median R_at_Tcool = {} Rvir'.format( np.nanmedian( R_at_Tcool ) / r_vir ) )
    
    # Don't do the below, because it's so far out that it's not visible
    # Create plot for volume filling distribution
#     n_rand = int( 1e5 )
#     data_coords = np.random.uniform( -r_vir, r_vir, (3, n_rand ) )
#     data_r = np.sqrt( ( data_coords ** 2. ).sum( axis=0 ) )
#     filtered_data_r = data_r[data_r<r_vir]
#     ax1.hist(
#         filtered_data_r,
#         bins = bins,
#         color = 'k',
#         density = True,
#         histtype = 'step',
#         linewidth = 3,
#         linestyle = '--',
#     )

    # ax.axvline(
    #     np.median( R_at_Tcool ),
    #     color = 'k',
    # )

    # ax1.axvline(
    #     w.r_gal[0],
    #     color = 'k',
    #     linestyle = '--',
    #     linewidth = 3,
    # )

    ax1.set_xlim( x_lim )

    ax1.set_xlabel( r'$R_{T=10^5{\rm K}}$ (kpc)', fontsize=22, labelpad=10 )
    ax1.xaxis.set_label_position( 'top' )
#     ax1.set_ylabel( 'count', fontsize=22, )

    ax1.tick_params( axis='x', top=True, labeltop=True, bottom=False, labelbottom=False )
    ax1.tick_params( axis='y', left=False, labelleft=False, )

    ax1.annotate(
        s='radius at which accreted gas cools',
        xy=(1,1),
        xycoords='axes fraction',
        xytext=(-10,-10),
        textcoords='offset points',
        ha = 'right',
        va = 'top',
        fontsize = 22,
    )

    
    ### FLOW PLOT ####
    
    ax2 = plt.subplot(gs[2:,0])
#     ax3 = plt.subplot(gs[6:,0])
    ax3 = ax
    
    # Choose particles
    particle_inds = np.random.choice( np.arange( w_j.n_particles ), size=n_particles, replace=False )
    
    # Get the time at the phase
    t_min_t_cool = ( t[:,np.newaxis] - t[inds] ).transpose()
    w_j.data_object.data['t_rel_t1e5'] = t_min_t_cool

    # Get positions in r-T space and color
    valid_value = ( t_min_t_cool < dt_after ) & ( t_min_t_cool > dt_before )
    valid_value = valid_value & ( w_j.get_data( 'n_out' ) == 0 )
    valid_value_inds = valid_value[particle_inds]
    r_vecs_all = w_j.get_data( 'R' )[particle_inds]
    T_vecs_all = w_j.get_data( 'T' )[particle_inds]
    K_vecs_all = w_j.get_data( 'entropy' )[particle_inds]
    
    # Plot quivers for each particles
    for k, particle_ind in enumerate( particle_inds ):
        
        # Format for quiver
        r_vecs = r_vecs_all[k][valid_value_inds[k]]
        T_vecs = np.log10( T_vecs_all[k][valid_value_inds[k]] )
        K_vecs = np.log10( K_vecs_all[k][valid_value_inds[k]] )
        x = r_vecs[1:]
        y = T_vecs[1:]
        y2 = K_vecs[1:]
        dx = r_vecs[:-1] - r_vecs[1:]
        dy = T_vecs[:-1] - T_vecs[1:]
        dy2 = K_vecs[:-1] - K_vecs[1:]
        angles_deg = np.arctan2( dy, dx ) * 180. / np.pi
        C = t_min_t_cool[particle_inds][k,valid_value_inds[k]][1:] * 1e3
        
        # Plot quiver
        quiver = ax2.quiver(
            x, y,
            dx, dy,
            C,
            angles = 'xy',
            units = 'y',
            scale = 10,
            minshaft = 2,
            headwidth = 2,
            headlength = 3.5,
#             color = 'red',
            cmap = palettable.scientific.diverging.Berlin_5_r.mpl_colormap,
            norm = plt.Normalize( -color_dt*1e3, color_dt*1e3 ),
        )
        plotting.add_colorbar(
            fig,
#             ax2,
            quiver,
            ax_location = [0.905, 0.125, 0.03, 0.6],
#             method = 'ax',
        )

#     w_j.plot_streamlines(
#         ax = ax2,
#         x_key = 'R',
#         y_key = 'logT',
#         start_ind = ind,
#         end_ind = 'time_based',
#         t_end = 5.,
#         sample_inds = particle_inds,
#         sample_selected_interval = False,
# #         x_data_kwargs = { 'smooth_data' : True, 'smoothing_window_length' : 7 },
# #         y_data_kwargs = { 'smooth_data' : True, 'smoothing_window_length' : 7 },
#         color = 'black',
#         fade_color = 'black',
#         min_fade_linewidth = 0.5,
#     #     fade_streamlines = False,
#     #     line_features = gas_to_star_line_features,
#         linewidth = 1,
#         x_label = 'R (kpc)',
#         y_label = 'logT (K)',
#     #     y_scale = 'log',
#         x_range = [ 0, 100. ],
#         y_range = [ 3.5, 6.9 ],
#     #     y_floor = 10.**3.8,
#     )

    # Plot reference entropy lines
    r = w_j.get_data( 'R' )
    at_border = ( r < 1.1 * x_lim[1] ) & ( r > 0.9 * x_lim[1] ) & valid_value
    k_at_border = w_j.get_data( 'entropy' )[at_border]
    med_k_at_border = np.nanmedian( k_at_border )
    a_vals = [ 0, 1, 2 ]
    r_arr = np.linspace( x_lim[0], x_lim[1], 256 )
    k_arrs = [ med_k_at_border * ( r_arr / x_lim[1] )**a for a in a_vals ]
    for m, k_arr in enumerate( k_arrs ):
        ax3.plot(
            r_arr,
            np.log10( k_arr ),
            color = '0.25',
            linewidth = 1.5,
#             linestyle = '--',
        )
        ax3.annotate(
            s = r'$\propto r^{' + str( a_vals[m] ) + r'}$',
            xy = ( r_arr[100], np.log10( k_arr[100] ) ),
            xycoords = 'data',
            xytext = ( 0, 0 ),
            textcoords = 'offset points',
            ha = 'right',
            va = 'bottom',
            fontsize = 24,
            color = '0.25',
        )

    # # Time
    # ax.plot(
    #     R_med,
    #     logT_med,
    #     linewidth = 3,
    #     color = 'b',
    # )
    # ax.fill_between(
    #     R_med,
    #     logT_low,
    #     logT_high,
    #     color = 'b',
    #     alpha = 0.25,
    # )
    
    for ax_k in [ ax2, ax3 ]:
        ax_k.annotate(
            s='accretion tracks',
            xy=(1,0),
            xycoords='axes fraction',
            xytext=(-10,10),
            textcoords='offset points',
            ha = 'right',
            va = 'bottom',
            fontsize = 22,
        )
    
    t_label = ax2.annotate(
        s = r'$t - t_{T=10^5 {\rm K}}$ (Myr)',
        xy = ( 1, 0 ),
        xycoords = 'axes fraction',
        xytext = ( 20, -30 ),
        textcoords = 'offset points',
        ha = 'center',
        va = 'top',
        fontsize = 24,
    )
    
    # 0.1 Rvir line
    for ax_k in [ ax1, ax2, ax3 ]:
        ax_k.axvline(
            0.05 * r_vir,
            color = 'k',
            linestyle = '--',
            linewidth = 3,
        )
        ax_k.axvline(
            r_vir,
            color = 'k',
            linestyle = '--',
            linewidth = 3,
        )
        ax_k.axvline(
            35.,
            color = 'k',
            linestyle = '--',
            linewidth = 3,
        )
        if ax_k.is_first_row():
            trans = matplotlib.transforms.blended_transform_factory( ax_k.transData, ax_k.transAxes )
            ax_k.annotate(
                s = r'$0.05 R_{\rm vir}$',
                xy = ( 0.05 * r_vir, 1.0 ),
                xycoords = trans,
                xytext = ( 6, -10 ),
                textcoords = 'offset points',
                ha = 'left',
                va = 'top',
                fontsize = 24,
            )
            ax_k.annotate(
                s = r'$ R_{\rm sonic}$',
                xy = ( 35., 1.0 ),
                xycoords = trans,
                xytext = ( 6, -10 ),
                textcoords = 'offset points',
                ha = 'left',
                va = 'top',
                fontsize = 24,
            )
        
    # Remove bottom ticks on middle axis
#     ax2.tick_params( axis='x', bottom=False, labelbottom=False )
    
    ax2.set_xlim( x_lim )
    ax2.set_ylim( np.log10( y_lim ) )
    ax2.set_ylabel( '$\log$T (K)', fontsize=22 )
    ax3.set_xlim( x_lim )
    ax3.set_ylim( np.log10( y2_lim) )
    ax2.set_xlabel( 'R (kpc)', fontsize=22 )
    ax3.set_ylabel( r'$\log$K (Kev cm$^2$)', fontsize=22 )
    
    plotting.save_fig(
        out_dir = '../figures',
        save_file = 'tracks_{}.pdf'.format( sim_name_j ),
        fig = fig,
    )


In [ ]:
t_half_width = 0.25
dt = 0.025
t_tphase_centers = np.arange( -t_half_width, t_half_width + 0.01, dt)
t_tphase_centers = np.array([ -0.15, -0.06, -0.03, 0., 0.03, 0.06, 0.15, ])

In [ ]:
all_phi_dists = {}
for j, ( sim_name_j, w_j ) in enumerate( w_set.items() ):
    
    w_j = w_j.data_object
    
    # Get the time at the phase
    R_at_Tcool, M_at_Tcool, L_at_Tcool, t_at_Tcool, inds = RMLt_at_Tcool[sim_name_j]
    t_tphase = ( t[:,np.newaxis] - t[inds] ).transpose()
    
    # Get phi (also known as theta)
    w_j.data_masker.clear_masks()
    w_j.data_masker.mask_data( 'PType', data_value=0 )
    phi = w_j.get_selected_data( 'Phi', compress=False )

    # Format data
    phi_flat = phi.flatten()
    t_tphase_flat = t_tphase.flatten()

    # Get distributions
    phi_dists = []
    for i, center in enumerate( t_tphase_centers ):
        bin_low = center - dt / 2.
        bin_high = center + dt / 2.
        in_bin = ( t_tphase_flat > bin_low ) & ( t_tphase_flat < bin_high )
        hist, bin_edges = np.histogram(
            np.cos( phi_flat[in_bin] * np.pi / 180 ),
            bins = np.linspace( -1., 1., 64 ),
            density = True,
        )
        phi_dists.append( hist )
    
    all_phi_dists[sim_name_j] = phi_dists

In [ ]:
bin_centers = bin_edges[:-1] + 0.5 * ( bin_edges[1] - bin_edges[0] )

In [ ]:
labeled_is = np.arange( len( t_tphase_centers ) )

In [ ]:
n_rows = len( sim_names )

fig = plt.figure( figsize=(10, 4.5*n_rows ), facecolor='w' )
main_ax = plt.gca()

gs = gridspec.GridSpec(n_rows, 1)
gs.update( hspace=0.001 )


z_max = t_tphase_centers.max()
z_min = t_tphase_centers.min()

for j, ( sim_name_j, w_j ) in enumerate( w_set.items() ):
    
    w_j = w_j.data_object
    
    phi_dists = all_phi_dists[sim_name_j]
    
    ax = plt.subplot(gs[j,0])

    for i, phi_dist in enumerate( phi_dists ):

        z_width = z_max - z_min
        color_value = ( t_tphase_centers[i] - z_min )/z_width
        color = palettable.scientific.diverging.Roma_3.mpl_colormap( color_value )

        if i in labeled_is:
            if np.isclose( t_tphase_centers[i], 0. ):
                t_tphase_centers[i] = 0
#             label = (
#                 r'$t - t_{T=10^5 {\rm K}}$ =' +
#                 ' {:.3g}'.format( t_tphase_centers[i]*1e3 ) +
#                 r' Myr'
#             )
            label = (
                '{:.3g}'.format( t_tphase_centers[i]*1e3 ) +
                r' Myr'
            )
        else:
            label = None

        line = ax.plot(
            bin_centers,
            phi_dist, #/ (np.pi / 180. / 2. * np.sin( bin_centers * np.pi/180. ) ),
            linewidth = 5,
            color = color,
            label = label,
#             zorder = 10 - i,
        )
        
    ax.tick_params(
        axis = 'x',
        top = True,
        labeltop = ax.is_first_row(),
        bottom = ax.is_last_row(),
        labelbottom = ax.is_last_row(),
    )

    ax.axhline(
        0.5,
        color = '.2',
        linestyle = '-',
        linewidth = 2,
    )
    ax.axvline(
        0,
        color = '.2',
        linestyle = '-',
        linewidth = 2,
    )
    
    # Sim name label
    ax.annotate(
        s = sim_name_j,
        xy = ( 0, 1 ),
        xycoords = 'axes fraction',
        xytext = ( 20, -20 ),
        textcoords = 'offset points',
        ha = 'left',
        va = 'top',
        fontsize = 26,
    )
    
    # Spherical line label
    if j == 1:
        ax.annotate(
            s = 'spherical\ndistribution',
            xy = ( 0.5, 0.5 ),
            xycoords = 'data',
            xytext = ( 0, 5 ),
            textcoords = 'offset points',
            ha = 'left',
            va = 'bottom',
            fontsize = 22,
        )

    if j == 0:
        t_label = ax.annotate(
            s = r'$t - t_{T=10^5 {\rm K}}$',
            xy = ( 1, 0.875 ),
            xycoords = 'axes fraction',
            xytext = ( -25, 0 ),
            textcoords = 'offset points',
            ha = 'right',
            va = 'bottom',
            fontsize = 24,
        )
        t_label.set_zorder( 1000 )
        ax.legend(
            prop={'size': 17},
            loc = 'center right',
        )

    ax.set_xlim( -1, 1 )
    ax.set_ylim( 0, 3.75 )

    ax.set_xlabel( r'$\cos\ \theta$', fontsize=22 )
    if ax.is_first_row():
        ax.xaxis.set_label_position( 'top' )
    ax.set_ylabel( r'PDF$\ (\cos\ \theta$)', fontsize=22 )

plotting.save_fig(
    out_dir = '../figures',
    save_file = 'theta_vs_t.pdf',
    fig = fig,
)

t_start = -4
t_end = 0.2

for j, ( sim_name_j, w_j_plotter ) in enumerate( w_set.items() ):
    
    print( 'Plotting luminosities for {}'.format( sim_name_j ) )
        
    # Mask stars
    w_j = w_j_plotter.data_object
    w_j.data_masker.clear_masks()
    w_j.data_masker.mask_data( 'PType', data_value=0, )

    fig = plt.figure( figsize=(10, 6 ), facecolor='w' )
    ax = plt.gca()

    # Get the time at the phase
    R_at_Tcool, M_at_Tcool, L_at_Tcool, t_at_Tcool, inds = RMLt_at_Tcool[sim_name]

    bins = np.linspace( t_start, t_end, 64 ),
    w_j_plotter.statistic_and_interval(
        't_rel_t1e5',
        'lcool',
        x_data = w_j.get_selected_data( 't_rel_t1e5' ).flatten(),
        y_data = w_j.get_selected_data( 'lcool' ).flatten(),
        ax = ax,
        bins = bins,
        color = palettable.cartocolors.qualitative.Vivid_2.mpl_colors[1],
        label = ''
    )
    w_j_plotter.statistic_and_interval(
        't_rel_t1e5',
        'lcompheat',
        x_data = w_j.get_selected_data( 't_rel_t1e5' ).flatten(),
        y_data = w_j.get_selected_data( 'lcompheat' ).flatten(),
        ax = ax,
        bins = bins,
        color = palettable.cartocolors.qualitative.Vivid_2.mpl_colors[0],
    )
    
    # Plot a reference line
    edot_comp = 4e49 * unyt.erg / unyt.yr
    spec_edot_comp = ( edot_comp / ( w.m_vir[600] * 0.8 * 0.05 / 0.25 * unyt.msun ) ).to( 'erg/g/s' )
    ax.axhline(
        spec_edot_comp,
        linestyle = '--',
        color = 'k',
        linewidth = 3,
    )
    ax.axhline(
        spec_edot_comp * 10,
        linestyle = '--',
        color = 'k',
        linewidth = 1,
    )
    ax.axhline(
        spec_edot_comp * 0.1,
        linestyle = '--',
        color = 'k',
        linewidth = 1,
    )
    
    # Labels
    ax.annotate(
        s = r'$L_{\rm cool}$',
        xy = ( 0, 1e-4 ),
        xycoords = 'data',
        color = palettable.cartocolors.qualitative.Vivid_2.mpl_colors[1],
        ha = 'left',
        va = 'bottom',
        fontsize = 22,
    )
    ax.annotate(
        s = r'$L_{\rm comp.\,heat}$',
        xy = ( 0, 0.1 ),
        xycoords = 'data',
        color = palettable.cartocolors.qualitative.Vivid_2.mpl_colors[0],
        ha = 'left',
        va = 'bottom',
        fontsize = 22,
    )

    ax.set_yscale( 'log' )
    ax.set_xlim( t_start, t_end )
    ax.set_xlabel( r'$t - t_{T=10^5 {\rm K}}$ (Gyr)', fontsize=22 )
    ax.set_ylabel( 'erg/g/s', fontsize=22 )
    
    plotting.save_fig(
        out_dir = '../figures',
        save_file = 'heating_vs_cooling_{}.pdf'.format( sim_name_j ),
        fig = fig,
    )